In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
from datetime import timedelta
import calendar

## 蚊蟲指數資料

In [ ]:
df_insect = pd.read_excel("03_三民區病媒蚊指數.xlsx", encoding = 'GB18030')
df_insect = df_insect.drop('LiNo', axis=1)
df_insect = df_insect.drop('latitude', axis=1)
df_insect = df_insect.drop('longitude', axis=1)
df_insect.head()

,year,ISO,Breteau,Aeg_aed_index,Res_index,Contain_index,Larvae_index
0,2005,11,2.000000,0.00,2.000000,3.571429,16.000000
1,2005,15,1.818182,0.00,1.818182,1.250000,1.818182
2,2005,19,2.000000,0.00,2.000000,1.960784,26.000000
3,2005,24,14.000000,0.00,8.000000,10.606061,410.000000
4,2005,26,4.000000,0.02,4.000000,4.081633,50.000000


In [ ]:
df_insect['year'] = df_insect['year'].astype(str)
df_insect['ISO'] = df_insect['ISO'].astype(str)
df_insect['index'] = df_insect['year'] + '-' + df_insect['ISO']
df_insect.head()

,year,ISO,Breteau,Aeg_aed_index,Res_index,Contain_index,Larvae_index,index
0,2005,11,2.000000,0.00,2.000000,3.571429,16.000000,2005-11
1,2005,15,1.818182,0.00,1.818182,1.250000,1.818182,2005-15
2,2005,19,2.000000,0.00,2.000000,1.960784,26.000000,2005-19
3,2005,24,14.000000,0.00,8.000000,10.606061,410.000000,2005-24
4,2005,26,4.000000,0.02,4.000000,4.081633,50.000000,2005-26


In [ ]:
df_insect = df_insect.groupby(by='index').mean()
df_insect.head()

,Breteau,Aeg_aed_index,Res_index,Contain_index,Larvae_index
index,,,,,
2005-10,1.487179,0.009744,1.538462,2.324706,15.846154
2005-11,2.333333,0.023333,2.111111,3.405051,38.888889
2005-13,3.538462,0.008462,1.000000,5.014069,7.461538
2005-14,1.000000,0.021333,0.866667,1.769701,14.133333
2005-15,2.132105,0.010974,1.950287,1.894605,30.787042


## 登革熱資料

In [ ]:
df_people = pd.read_excel("03_三民區登革熱病例.xlsx", encoding = 'GB18030')
df_people.head()

,居住縣市,居住鄉鎮,值,居住村里,值.1,發病年齡(足歲),性別,職業,發病年月,發病日期(民國-yyy),...,通報疾病,確定病名,住院情況,是否採檢體,是否死亡,是否境外移入,是否參加旅行團,感染國家,主要症狀LIST,其他症狀
0,高雄市,三民區,1,力行里,1,70,女,無,9511,95,...,登革熱,登革熱,是,是,否,否,NaN,中華民國,"倦怠,發燒(38℃以上),頭痛,其他",喉嚨痛、個案是糖尿病患
1,高雄市,三民區,1,力行里,1,47,女,無,9511,95,...,登革熱,登革熱,是,是,否,否,NaN,中華民國,"倦怠,發燒(38℃以上),頭痛,關節痛,其他",喉嚨痛
2,高雄市,三民區,1,力行里,1,49,男,無,9807,98,...,登革熱,NaN,是,是,否,否,NaN,中華民國,"口乾,肌肉痛,食慾不振,發燒(38℃以上),頭痛,關節痛,其他","心臟病,H/T.CVA.DM.WBC2700,PLT144000,GOT47,GPT36"
3,高雄市,三民區,1,力行里,1,13,女,學生,9811,98,...,登革熱,NaN,NaN,是,否,否,NaN,中華民國,"紅疹,倦怠,發燒(38℃以上),其他",喉嚨痛
4,高雄市,三民區,1,力行里,1,65,女,教育服務業,9812,98,...,登革熱,NaN,否,是,否,否,NaN,中華民國,"咳嗽,發燒(38℃以上),其他","呼吸喘,洗腎"


In [ ]:
df_people.columns

Index(['居住縣市', '居住鄉鎮', '值', '居住村里', '值.1', '發病年齡(足歲)', '性別', '職業', '發病年月',
       '發病日期(民國-yyy)', '發病日期(民國-yyymmdd)', '周ISO-8601', '診斷日期(民國-yyymmdd)',
       '報告日期(民國-yyymmdd)', '衛生局收到日期(民國-yyymmdd)', '管制署收到日期(民國-yyymmdd)',
       '通報醫療院所名稱', '通報醫療院所縣市', '通報醫療院所鄉鎮', '通報疾病', '確定病名', '住院情況', '是否採檢體',
       '是否死亡', '是否境外移入', '是否參加旅行團', '感染國家', '主要症狀LIST', '其他症狀'],
      dtype='object')

In [ ]:
df_people = df_people[['發病日期(民國-yyymmdd)', '發病日期(民國-yyy)', '周ISO-8601','是否境外移入', '是否參加旅行團']] 
df_people.head()

,發病日期(民國-yyymmdd),發病日期(民國-yyy),周ISO-8601,是否境外移入,是否參加旅行團
0,951123,95,47,否,NaN
1,951127,95,48,否,NaN
2,980715,98,29,否,NaN
3,981121,98,47,否,NaN
4,981203,98,49,否,NaN


In [ ]:
df_people['year'] = df_people['發病日期(民國-yyy)'] + 1911
df_people.head()

,發病日期(民國-yyymmdd),發病日期(民國-yyy),周ISO-8601,是否境外移入,是否參加旅行團,year
0,951123,95,47,否,NaN,2006
1,951127,95,48,否,NaN,2006
2,980715,98,29,否,NaN,2009
3,981121,98,47,否,NaN,2009
4,981203,98,49,否,NaN,2009


In [ ]:
#df_people['發病日期(民國-yyymmdd)'] = df_people['發病日期(民國-yyymmdd)'].astype(str)

In [ ]:
df_people['year'] = df_people['year'].astype(str)
df_people['周ISO-8601'] = df_people['周ISO-8601'].astype(str)
df_people['index'] = df_people['year'] + '-' + df_people['周ISO-8601']
#df_people['index'] = df_people['發病日期(民國-yyymmdd)'][:2] + '-' + df_people['發病日期(民國-yyymmdd)'][2:4] + '-' + df_people['發病日期(民國-yyymmdd)'][4:]
#df_people['index'] = pd.to_datetime(df_people['index'])
#df_people['week'] = df_people['index'].dt.weekofyear
#df_people['index'] = df_people['index'].apply(lambda x:time.strftime("%Y-%U", time.localtime()))
df_people.head()

,發病日期(民國-yyymmdd),發病日期(民國-yyy),周ISO-8601,是否境外移入,是否參加旅行團,year,index
0,951123,95,47,否,NaN,2006,2006-47
1,951127,95,48,否,NaN,2006,2006-48
2,980715,98,29,否,NaN,2009,2009-29
3,981121,98,47,否,NaN,2009,2009-47
4,981203,98,49,否,NaN,2009,2009-49


In [ ]:
df_people.loc[df_people['是否境外移入']=='是', '是否境外移入'] = 1
df_people.loc[df_people['是否境外移入']=='否', '是否境外移入'] = 0

In [ ]:
ind = df_people['index'].unique()
travel_list = []
for i in ind:
    t = np.sum(df_people[df_people['index']==i]['是否境外移入'])
    travel_list.append(t)

In [ ]:
number = pd.value_counts(df_people['index'])
df_people_clean = pd.DataFrame({'index':number.index, 'number':number.values, 'travel':travel_list})
df_people_clean.sort_values(by=['index'], inplace = True)
df_people_clean.index = df_people_clean['index']
df_people_clean = df_people_clean.drop('index', axis=1)
df_people_clean.head()

,number,travel
index,,
2005-12,1,0
2005-14,1,0
2005-18,2,1
2005-20,1,0
2005-23,1,0


In [ ]:
df_people_clean

,number,travel
index,,
2005-12,1,0
2005-14,1,0
2005-18,2,1
2005-20,1,0
2005-23,1,0
...,...,...
2012-51,4,0
2012-52,5,0
2012-6,2,0


## 雨量資料

In [ ]:
weather = pd.read_csv('雨量估計.csv',encoding = 'ANSI')
item = list(pd.unique(weather[weather['time']=='1-Jan-05']['item']))
weather.head()

,time,station,item,h01,h02,h03,h04,h05,h06,h07,...,h17,h18,h19,h20,h21,h22,h23,h24,area_code,city
0,1-Jan-05,小港,AMB_TEMP,11.30,11.03,10.98,10.59,10.270,10.040,10.080,...,15.880,15.190,14.720,14.510,13.870,13.170,12.810,12.730,211.0,小港區
1,1-Jan-05,小港,CH4,NaN,NaN,NaN,NaN,1.806,1.794,1.775,...,1.731,1.732,1.731,1.750,1.739,1.776,1.775,1.740,211.0,小港區
2,1-Jan-05,小港,CO,0.53,0.51,NaN,0.47,0.510,0.510,0.590,...,0.630,0.840,0.970,1.060,0.930,1.100,1.150,0.900,211.0,小港區
3,1-Jan-05,小港,NMHC,NaN,NaN,NaN,NaN,0.042,0.030,0.077,...,0.060,0.181,0.296,0.409,0.311,0.419,0.486,0.752,211.0,小港區
4,1-Jan-05,小港,NO,4.31,3.11,3.30,2.28,5.530,NaN,20.150,...,3.590,3.220,4.870,18.230,8.770,34.540,59.980,82.970,211.0,小港區


In [ ]:
weather = weather[weather["item"].isin(['AMB_TEMP', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10',
                                        'PM2.5', 'RAINFALL', 'RH', 'SO2', 'THC', 'UVB', 'WIND_DIR','WIND_SPE'])]

In [ ]:
weather['year'] = (np.array(weather['time'].str.split('-').tolist())[:,2]).astype(str)
weather['year'] = '20' + weather['year'] 
weather['month'] = (np.array(weather['time'].str.split('-').tolist())[:,1]).astype(str)
weather['day'] = (np.array(weather['time'].str.split('-').tolist())[:,0]).astype(str)
weather = weather.drop('time',axis = 1)
weather.head()

,station,item,h01,h02,h03,h04,h05,h06,h07,h08,...,h20,h21,h22,h23,h24,area_code,city,year,month,day
0,小港,AMB_TEMP,11.30,11.03,10.98,10.59,10.270,10.040,10.080,10.070,...,14.510,13.870,13.170,12.810,12.730,211.0,小港區,2005,Jan,1
1,小港,CH4,NaN,NaN,NaN,NaN,1.806,1.794,1.775,1.751,...,1.750,1.739,1.776,1.775,1.740,211.0,小港區,2005,Jan,1
2,小港,CO,0.53,0.51,NaN,0.47,0.510,0.510,0.590,0.650,...,1.060,0.930,1.100,1.150,0.900,211.0,小港區,2005,Jan,1
3,小港,NMHC,NaN,NaN,NaN,NaN,0.042,0.030,0.077,0.134,...,0.409,0.311,0.419,0.486,0.752,211.0,小港區,2005,Jan,1
4,小港,NO,4.31,3.11,3.30,2.28,5.530,NaN,20.150,22.970,...,18.230,8.770,34.540,59.980,82.970,211.0,小港區,2005,Jan,1


In [ ]:
month = list(calendar.month_abbr)
month.remove('')
weather['month'] = weather['month'].replace(month, list(np.arange(12)+1))
weather['month'] = weather['month'].astype(str)
weather['time'] = weather['year'] + '-' + weather['month'].str.zfill(2) + '-' + weather['day'].str.zfill(2)
weather['time'] = pd.to_datetime(weather['time'], errors='coerce')
weather.head()

,station,item,h01,h02,h03,h04,h05,h06,h07,h08,...,h21,h22,h23,h24,area_code,city,year,month,day,time
0,小港,AMB_TEMP,11.30,11.03,10.98,10.59,10.270,10.040,10.080,10.070,...,13.870,13.170,12.810,12.730,211.0,小港區,2005,1,1,2005-01-01
1,小港,CH4,NaN,NaN,NaN,NaN,1.806,1.794,1.775,1.751,...,1.739,1.776,1.775,1.740,211.0,小港區,2005,1,1,2005-01-01
2,小港,CO,0.53,0.51,NaN,0.47,0.510,0.510,0.590,0.650,...,0.930,1.100,1.150,0.900,211.0,小港區,2005,1,1,2005-01-01
3,小港,NMHC,NaN,NaN,NaN,NaN,0.042,0.030,0.077,0.134,...,0.311,0.419,0.486,0.752,211.0,小港區,2005,1,1,2005-01-01
4,小港,NO,4.31,3.11,3.30,2.28,5.530,NaN,20.150,22.970,...,8.770,34.540,59.980,82.970,211.0,小港區,2005,1,1,2005-01-01


In [ ]:
hour = []
for i in range(24):
    name = 'h' + str(i +1).zfill(2)
    hour.append(name)

In [ ]:
weather['iso'] = weather['time'].dt.weekofyear
weather[hour] = weather[hour].astype('float32')
weather['sum'] = np.sum(weather[hour],1)
weather = weather.drop(hour,axis = 1)
weather.head()

,station,item,area_code,city,year,month,day,time,iso,sum
0,小港,AMB_TEMP,211.0,小港區,2005,1,1,2005-01-01,53,310.779999
1,小港,CH4,211.0,小港區,2005,1,1,2005-01-01,53,35.184998
2,小港,CO,211.0,小港區,2005,1,1,2005-01-01,53,17.490000
3,小港,NMHC,211.0,小港區,2005,1,1,2005-01-01,53,4.790000
4,小港,NO,211.0,小港區,2005,1,1,2005-01-01,53,453.280029


In [ ]:
weather['iso'] = weather['iso'].astype(int)
weather['iso'] = weather['iso'].astype(str)
weather['index'] = weather['year'] + '-' + weather['iso']


In [ ]:
i = 0
item.remove('WIND_DIR')
for name in item:
    wether_name = weather[weather['item'] == name].groupby("index").sum()
    if i==0:
        index = pd.unique(weather['index'])
        df_weather = pd.DataFrame(columns=['AMB_TEMP'], data =np.array(wether_name['sum']),index = index)
        i = i+1
    else:
        df_weather[name] = wether_name['sum']
df_weather.head()

,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,UVB,WIND_SPE
2005-53,39359.691406,340.748016,406.100006,66.386002,6723.830078,17423.250000,24146.919922,9454.500000,54737.0,25961.0,0.0,31381.740234,6802.299805,407.134003,29.000000,1202.900024
2005-1,39324.128906,1189.997070,1615.280029,344.054016,31692.089844,69834.593750,101526.781250,30121.298828,211148.0,100171.0,4.4,140023.046875,28720.798828,1534.038940,121.500000,4042.100098
2005-2,42410.628906,1144.978027,1343.030029,333.666016,24532.019531,56909.480469,81441.382812,26011.000000,180874.0,93448.0,7.0,140920.000000,23939.400391,1478.641968,138.800003,5350.740234
2005-3,46519.250000,905.453003,1751.270020,226.852997,29608.958984,65394.167969,95002.687500,45185.199219,246740.0,125944.0,0.8,137519.296875,26496.300781,1132.322998,167.199997,3933.879883
2005-4,47734.898438,975.057983,1684.359985,219.326004,29877.839844,58273.718750,88151.328125,39355.898438,197063.0,100961.0,7.8,144511.093750,22552.099609,1194.377930,181.000000,4034.500000


## 資料合併

In [ ]:
data = pd.concat([df_insect, df_people_clean, df_weather], axis=1, join_axes=[df_insect.index])
data

C:\Users\User\Anaconda3\envs\tf2_rpy2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  """Entry point for launching an IPython kernel.


,Breteau,Aeg_aed_index,Res_index,Contain_index,Larvae_index,number,travel,AMB_TEMP,CH4,CO,...,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,UVB,WIND_SPE
index,,,,,,,,,,,,,,,,,,,,,
2005-10,1.487179,0.009744,1.538462,2.324706,15.846154,NaN,NaN,53710.011719,1191.270996,1490.229980,...,83933.601562,44628.101562,178303.0,95307.0,197.199997,136157.453125,20731.199219,1421.073975,152.000000,4038.570068
2005-11,2.333333,0.023333,2.111111,3.405051,38.888889,NaN,NaN,32959.511719,1234.130981,1792.500000,...,88407.031250,67071.703125,277692.0,131294.0,1.200000,139997.234375,25277.099609,1504.006958,169.500000,3680.689941
2005-13,3.538462,0.008462,1.000000,5.014069,7.461538,NaN,NaN,57583.589844,1124.965942,1335.869995,...,68874.953125,52936.800781,142121.0,74474.0,102.599998,146081.484375,23208.300781,1332.848022,236.199997,4680.959961
2005-14,1.000000,0.021333,0.866667,1.769701,14.133333,1.0,0.0,54008.640625,1189.927002,1101.630005,...,50068.449219,56204.500000,169800.0,73301.0,115.800003,136098.296875,19887.099609,1331.514038,266.500000,4518.709961
2005-15,2.132105,0.010974,1.950287,1.894605,30.787042,NaN,NaN,55734.128906,1238.938965,1561.270020,...,72074.601562,70654.796875,201684.0,101087.0,24.600000,144244.656250,23087.000000,1444.128052,185.000000,3887.649902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-50,102.181818,0.009091,35.212121,6.713792,486.060606,3.0,0.0,42995.000000,3364.399902,1290.300049,...,64401.300781,56554.300781,158566.0,91594.0,0.000000,149621.000000,13381.900391,3937.899902,176.199997,3395.500000
2012-6,0.892148,0.000000,0.760995,1.289542,3.768108,2.0,0.0,55349.000000,3242.699951,1255.349976,...,64244.800781,33468.601562,159634.0,98507.0,214.199997,160316.000000,9209.500000,3762.500000,231.000000,4530.299805
2012-7,0.829993,0.000102,0.772360,1.402307,12.923461,2.0,0.0,53362.000000,3175.800049,1324.410034,...,61589.199219,50776.500000,190388.0,112591.0,204.600006,144488.000000,11662.200195,3690.699951,229.199997,4137.700195


In [ ]:
data['number'] = data['number'].fillna(0)
data.head()

,Breteau,Aeg_aed_index,Res_index,Contain_index,Larvae_index,number,travel,AMB_TEMP,CH4,CO,...,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,UVB,WIND_SPE
index,,,,,,,,,,,,,,,,,,,,,
2005-10,1.487179,0.009744,1.538462,2.324706,15.846154,0.0,NaN,53710.011719,1191.270996,1490.229980,...,83933.601562,44628.101562,178303.0,95307.0,197.199997,136157.453125,20731.199219,1421.073975,152.000000,4038.570068
2005-11,2.333333,0.023333,2.111111,3.405051,38.888889,0.0,NaN,32959.511719,1234.130981,1792.500000,...,88407.031250,67071.703125,277692.0,131294.0,1.200000,139997.234375,25277.099609,1504.006958,169.500000,3680.689941
2005-13,3.538462,0.008462,1.000000,5.014069,7.461538,0.0,NaN,57583.589844,1124.965942,1335.869995,...,68874.953125,52936.800781,142121.0,74474.0,102.599998,146081.484375,23208.300781,1332.848022,236.199997,4680.959961
2005-14,1.000000,0.021333,0.866667,1.769701,14.133333,1.0,0.0,54008.640625,1189.927002,1101.630005,...,50068.449219,56204.500000,169800.0,73301.0,115.800003,136098.296875,19887.099609,1331.514038,266.500000,4518.709961
2005-15,2.132105,0.010974,1.950287,1.894605,30.787042,0.0,NaN,55734.128906,1238.938965,1561.270020,...,72074.601562,70654.796875,201684.0,101087.0,24.600000,144244.656250,23087.000000,1444.128052,185.000000,3887.649902


In [ ]:
data['travel'][data['travel'].isna()] = 0

C:\Users\User\Anaconda3\envs\tf2_rpy2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
data.head()

,Breteau,Aeg_aed_index,Res_index,Contain_index,Larvae_index,number,travel,AMB_TEMP,CH4,CO,...,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,UVB,WIND_SPE
index,,,,,,,,,,,,,,,,,,,,,
2005-10,1.487179,0.009744,1.538462,2.324706,15.846154,0.0,0.0,53710.011719,1191.270996,1490.229980,...,83933.601562,44628.101562,178303.0,95307.0,197.199997,136157.453125,20731.199219,1421.073975,152.000000,4038.570068
2005-11,2.333333,0.023333,2.111111,3.405051,38.888889,0.0,0.0,32959.511719,1234.130981,1792.500000,...,88407.031250,67071.703125,277692.0,131294.0,1.200000,139997.234375,25277.099609,1504.006958,169.500000,3680.689941
2005-13,3.538462,0.008462,1.000000,5.014069,7.461538,0.0,0.0,57583.589844,1124.965942,1335.869995,...,68874.953125,52936.800781,142121.0,74474.0,102.599998,146081.484375,23208.300781,1332.848022,236.199997,4680.959961
2005-14,1.000000,0.021333,0.866667,1.769701,14.133333,1.0,0.0,54008.640625,1189.927002,1101.630005,...,50068.449219,56204.500000,169800.0,73301.0,115.800003,136098.296875,19887.099609,1331.514038,266.500000,4518.709961
2005-15,2.132105,0.010974,1.950287,1.894605,30.787042,0.0,0.0,55734.128906,1238.938965,1561.270020,...,72074.601562,70654.796875,201684.0,101087.0,24.600000,144244.656250,23087.000000,1444.128052,185.000000,3887.649902


In [ ]:
data.to_csv('Mosquito.csv')